In [1]:
import os
# Changes the current working directory so to mimic running from projects root, i.e (./yolo_v1_taco).
# os.getcwd().rsplit("/", 1)[0]
os.chdir(os.getcwd().rsplit("/", 1)[0])
print(os.getcwd())

/Users/tonyavis/Main/AI_public_projects/object_detection/yolo_v1_taco


In [2]:
from utils.load_config import load_config

In [ ]:
import logging
# TODO implement a basic log to the project, but dont have to go to deep, just learn for future projects.
# === 1. Configure the logger ===
logging.basicConfig(
    level=logging.INFO,  # Set default level (DEBUG, INFO, WARNING, ERROR)
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler("test.log"),    # Logs to a file
        logging.StreamHandler()              # Logs to the console
    ]
)

# === 2. Usage ===
logging.info("Starting training loop...")
logging.warning("GPU memory almost full!")
logging.error("Model checkpoint failed to save")
logging.debug("This is useful for debugging internals, won't show unless level=DEBUG")

2025-07-08 18:30:55,799 | INFO | Starting training loop...
2025-07-08 18:30:55,799 | WARNING | GPU memory almost full!
2025-07-08 18:30:55,800 | ERROR | Model checkpoint failed to save
